# Инструменты для работы с текстом

Анализ текстовых данных - это отдельное направление, здесь будет совсем небольшое введение.
С текстовыми данными можно решать как задачи обучения с учителем (классификация текстов), так и задачу обучения без учителя (кластеризация).

Предобработка текста

Первый шаг любой аналитики – получение данных. Предположим, что данные представляются собой набор текстов. Все известные нам алгоритмы работают не в текстами, а с объектами, которые описываются вектором признаков (чаще всего численных, категориальные мы умеем преобразовывать). Что делать, если наши объекты - это текст? 

Следующая после получения данных задача: предобработка. Основная цель предобработки: преобразовать текстовые данные в удобный для построения модели вид.

Базовые шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. удаление пунктуации
5. фильтрация по частоте/длине/соответствию регулярному выражению
6. лемматизация или стемминг
7. векторизация (эмбеддинг)

Чаще всего применяются все эти шаги, но в разных задачах какие-то могут опускаться, поскольку приводят к потере информации

In [ ]:
import pandas as pd
import numpy as np

from sklearn.metrics import * 
from sklearn.model_selection import train_test_split 

* Сейчас мы попробуем получить преобразование предложений в численный вектор, с которым может работать стандартный алгоритм машинного обучения. 
* Для этого нам понадобится познакомиться с понятием n-gram - самых мелких элементов предложения, с которыми можно работать. 
* Подсчитав количество этих n-грам в предложениях, мы получим искомые численные представления.

## n-граммы

Самые мелкие структуры языка, с которыми мы работаем, называются **n-граммами**.
У n-граммы есть параметр n - количество слов, которые попадают в такое представление текста.
* Если n = 1 - то мы смотрим на то, сколько раз каждое слово встретилось в тексте. Получаем _униграммы_
* Если n = 2 - то мы смотрим на то, сколько раз каждая пара подряд идущих слов, встретилась в тексте. Получаем _биграммы_

Функция для работы с n-граммами реализована в библиотке **nltk** (Natural Language ToolKit), импортируем эту функцию: 

In [ ]:
from nltk import ngrams

Прежде чем получать n-граммы, нужно разделить предложение на отдельные слова.  Для этого используем метод ```split()```.

In [ ]:
sentence = 'Кто же победит на выборах в США: Трамп или Байден?'

In [ ]:
sentence_split = sentence.split()
sentence_split

['Кто',
 'же',
 'победит',
 'на',
 'выборах',
 'в',
 'США:',
 'Трамп',
 'или',
 'Байден?']

Кажется, что нам тут мешают знаки препинания. Дайвайте от них избавимся. 

In [ ]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
for ch in string.punctuation:
  sentence = sentence.replace(ch,"")

In [ ]:
sentence

'Кто же победит на выборах в США Трамп или Байден'

In [ ]:
sentence_split = sentence.split()

In [ ]:
sentence_split

['Кто', 'же', 'победит', 'на', 'выборах', 'в', 'США', 'Трамп', 'или', 'Байден']

Чтобы получить n-грамму для такой последовательности, используем функцию ```ngrams()```. 

На вход передается два параметра:
* лист с разделенным на отдельные слова предложением (у нас он хранится в переменной ```sent```);
* параметр n, определяющий, какой тип n-грамм мы хотим получить.


Чтобы полученный объект отобразить, делаем из него ```list```. 

In [ ]:
list(ngrams(sentence_split, 1)) # униграммы

[('Кто',),
 ('же',),
 ('победит',),
 ('на',),
 ('выборах',),
 ('в',),
 ('США',),
 ('Трамп',),
 ('или',),
 ('Байден',)]

Аналогично мы можем получить биграммы - для этого заменяем параметр **n** в функции **ngrams** с 1 на 2.

In [ ]:
list(ngrams(sentence_split, 2)) # биграммы

[('Кто', 'же'),
 ('же', 'победит'),
 ('победит', 'на'),
 ('на', 'выборах'),
 ('выборах', 'в'),
 ('в', 'США'),
 ('США', 'Трамп'),
 ('Трамп', 'или'),
 ('или', 'Байден')]

In [ ]:
list(ngrams(sentence_split, 3)) # триграммы

[('Кто', 'же', 'победит'),
 ('же', 'победит', 'на'),
 ('победит', 'на', 'выборах'),
 ('на', 'выборах', 'в'),
 ('выборах', 'в', 'США'),
 ('в', 'США', 'Трамп'),
 ('США', 'Трамп', 'или'),
 ('Трамп', 'или', 'Байден')]

In [ ]:
list(ngrams(sentence_split, 5)) # ... пентаграммы

[('Кто', 'же', 'победит', 'на', 'выборах'),
 ('же', 'победит', 'на', 'выборах', 'в'),
 ('победит', 'на', 'выборах', 'в', 'США'),
 ('на', 'выборах', 'в', 'США', 'Трамп'),
 ('выборах', 'в', 'США', 'Трамп', 'или'),
 ('в', 'США', 'Трамп', 'или', 'Байден')]

## Векторизаторы

Векторизатор преобразует слово или набор слов в числовой вектор, понятный алгоритму машинного обучения, который привык работать с числовыми табличными данными.

Ниже - пример преобразования слов в двумерных вектор, каждому слову соответствует точка на плоскости.

<a href="https://drive.google.com/uc?id=1ukv-FTj0jeVdcgVlOaNBocUfNuYGGVZg
" target="_blank"><img src="https://drive.google.com/uc?id=1ukv-FTj0jeVdcgVlOaNBocUfNuYGGVZg" 
alt="IMAGE ALT TEXT HERE" width="600" border="0" /></a>

На начальном этапе нам будет достаточно тех инструментов, которые уже есть в библиотеке **sklearn**.

In [ ]:
from sklearn.tree import DecisionTreeClassifier # можно заменить на другой классификатор
from sklearn.naive_bayes import MultinomialNB # наивный байесовский классификатор
from sklearn.feature_extraction.text import CountVectorizer # модель "мешка слов", см. далее

Самый простой способ извлечь признаки из текстовых данных -- векторизаторы: `CountVectorizer` и `TfidfVectorizer`

Объект `CountVectorizer` делает следующую вещь:
* строит для каждого документа (каждой пришедшей ему строки) вектор размерности `n`, где `n` -- количество слов или n-грам во всём корпусе
* заполняет каждый i-тый элемент количеством вхождений слова в данный документ

<a href="https://drive.google.com/uc?id=1ukv-FTj0jeVdcgVlOaNBocUfNuYGGVZg
" target="_blank"><img src="https://drive.google.com/uc?id=1jHmkrGZTMawM46Yzxh243Ur1y5pYKzrl" 
alt="IMAGE ALT TEXT HERE" width="600" border="0" /></a>

На рисунке пример векторизации для униграмм, но можно использовать любые n-граммы. Для этого у объекта ```CountVectorizer()``` есть параметр **ngram_range**, который отвечает за то, какие n-граммы мы используем в качестве признаов:<br/>
ngram_range=(1, 1) -- униграммы<br/>
ngram_range=(3, 3) -- триграммы<br/>
ngram_range=(1, 3) -- униграммы, биграммы и триграммы.

## Пример

К сожалению, на русском языке всё ещё очень мало годных наборов данных. Набор данных нашёл тут: https://github.com/sismetanin/rureviews

In [ ]:
!pip install PyDrive

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':"1j-DhO_XD5EqzOqVSR4Wz1kNzhLmpeTkZ"}) 
downloaded.GetContentFile('women-clothing-accessories.csv')

In [ ]:
data = pd.read_csv('women-clothing-accessories.csv', sep='\t', usecols=[0, 1])

In [ ]:
data.head()

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative


In [ ]:
data.sample(10)

,review,sentiment
9700,"товар не пришел, деньги не вернули. продавец в...",negative
78254,"куртка пришла быстро, хорошего качества и без ...",positive
69048,"Очень теплый кардиган, крупная вязка, ощущаетс...",positive
29445,"заказала размер М, по факту размер S. Рукава к...",negative
27592,"товар не пришел, деньги вернули",negative
25837,"Мятый, не держит, получила быстро",negative
26702,Качество пошива нормальное. Но!!! Пуховик выпо...,negative
74390,Платье прикольное! Шло до Свердл.обл. ровно 1 ...,positive
36988,"товар не пришел,вернули деньги.",neautral
15496,Очень криво сшито. одна половина длиннее другой,negative


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data.review, data.sentiment, train_size = 0.7)

In [ ]:
data.sentiment.value_counts()

neautral    30000
negative    30000
positive    30000
Name: sentiment, dtype: int64

In [ ]:
y_train.value_counts()

negative    21034
positive    21019
neautral    20946
Name: sentiment, dtype: int64

Инициализируем `CountVectorizer()`, указав в качестве признаков униграммы:

In [ ]:
vectorizer = CountVectorizer(ngram_range=(1, 1))

После инициализации _vectorizer_ можно обучить на наших данных. 

Для обучения используем обучающую выборку ```x_train```, но в отличие от классификатора мы используем метод ```fit_transform()```: сначала обучаем наш векторизатор, а потом сразу применяем его к нашему набору данных. Это похоже на то, как мы работали с one-hot-encoderом.

In [ ]:
vectorized_x_train = vectorizer.fit_transform(x_train)

Так как результат не зависит от порядка слов в текстах, то говорят, что такая модель представления текстов в виде векторов получается из *гипотезы представления текста как мешка слов*

В `vectorizer.vocabulary_` лежит словарь, отображение слов в их индексы:

In [ ]:
list(vectorizer.vocabulary_.items())[:10]

[('пришёл', 31551),
 ('мал', 18011),
 ('материал', 18269),
 ('ожидался', 23477),
 ('другой', 11224),
 ('немного', 20996),
 ('прозрачный', 32010),
 ('товар', 39618),
 ('пришел', 31484),
 ('менее', 18455)]

In [ ]:
vectorized_x_train.shape

(62999, 44490)

In [ ]:
x_train.shape

(62999,)

Так как теперь у нас есть **численное представление** и набор входных признаков, то мы можем обучить нашу модель

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(vectorized_x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
clf2 = MultinomialNB()
clf2.fit(vectorized_x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

С тестовыми данными нужно проделать то же самое, что и с данными для обучения: сделать из текстов вектора, которые можно передавать в классификатор для прогноза класса объекта. 

У нас уже есть обученный векторизатор ```vectorizer```, поэтому используем метод ```transform()``` (просто применить его), а не ```fit_transform``` (обучить и применить).

In [ ]:
vectorized_x_test = vectorizer.transform(x_test)

Как раньше, для получения прогноза у обученного классификатора используем метод ```predict()```.

С помощью функции ```classification_report()```, которая считает сразу несколько метрик качества классификации, посмотрим на то, насколько хорошо мы предсказываем положительную или отрицательную тональность твита .

In [ ]:
pred = clf.predict(vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    neautral       0.52      0.53      0.52      9054
    negative       0.63      0.63      0.63      8966
    positive       0.76      0.75      0.76      8981

    accuracy                           0.64     27001
   macro avg       0.64      0.64      0.64     27001
weighted avg       0.64      0.64      0.64     27001



In [ ]:
pred2 = clf2.predict(vectorized_x_test)
print(classification_report(y_test, pred2))

              precision    recall  f1-score   support

    neautral       0.59      0.66      0.62      9054
    negative       0.72      0.62      0.67      8966
    positive       0.84      0.85      0.84      8981

    accuracy                           0.71     27001
   macro avg       0.72      0.71      0.71     27001
weighted avg       0.72      0.71      0.71     27001



Итак, наивный байесовский классификатор легко побил дерево решений. Дальше работаем с ним.

### Отступление: F-мера

Прошлый раз мы разобрали метрики качества классификации, которые выводятся из матрицы ошибок (confision matrix). 

**Полнота** (Sensitivity, True Positive Rate, Recall, Hit Rate) отражает какой процент объектов положительного класса мы правильно классифицировали.

**Точность** (Precision, Positive Predictive Value) отражает какой процент положительных объектов (т.е. тех, что мы считаем положительными) правильно классифицирован. (Не путать с Accuracy!)

Легко построить алгоритм со 100%-й полнотой: он все объекты относит к классу 1, но при этом точность может быть очень низкой. Нетрудно построить алгоритм с близкой к 100% точностью: он относит к классу 1 только те объекты, в которых уверен, при этом полнота может быть низкая.

**F1-мера** (F1 score) является средним гармоническим точности и полноты, максимизация этого функционала приводит к одновременной максимизации этих двух «ортогональных критериев»

$$F_1 = \frac{2}{\mathrm{recall}^{-1} + \mathrm{precision}^{-1}} = 2 \cdot \frac{\mathrm{precision} \cdot \mathrm{recall}}{\mathrm{precision} + \mathrm{recall}} = \frac{\mathrm{tp}}{\mathrm{tp} + \frac12 (\mathrm{fp} + \mathrm{fn}) } $$

Также рассматривают весовое среднее гармоническое точности и полноты –  $F_\beta$-меру:

$$F_\beta = (1 + \beta^2) \cdot \frac{\mathrm{precision} \cdot \mathrm{recall}}{(\beta^2 \cdot \mathrm{precision}) + \mathrm{recall}} = \frac {(1 + \beta^2) \cdot \mathrm{tp} }{(1 + \beta^2) \cdot \mathrm{tp} + \beta^2 \cdot \mathrm{fn} + \mathrm{fp}}\,$$

Изменение $\beta$ позволяет делать один из критериев (точность или полноту) важнее при оптимизации.

## Биграммы

Попробуем сделать то же самое, используя в качестве признаков униграммы и биграммы:

In [ ]:
# инициализируем векторайзер 
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2))

In [ ]:
# обучаем его и сразу применяем к x_train
bigram_vectorized_x_train = bigram_vectorizer.fit_transform(x_train)

In [ ]:
# инициализируем и обучаем классификатор
clf = MultinomialNB()
clf.fit(bigram_vectorized_x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
# применяем обученный векторизатор к тестовым данным
bigram_vectorized_x_test = bigram_vectorizer.transform(x_test)

In [ ]:
# получаем предсказания и выводим информацию о качестве
pred = clf.predict(bigram_vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    neautral       0.61      0.67      0.64      9054
    negative       0.72      0.66      0.69      8966
    positive       0.87      0.86      0.87      8981

    accuracy                           0.73     27001
   macro avg       0.74      0.73      0.73     27001
weighted avg       0.74      0.73      0.73     27001



У меня получилось повысить точность на пару процентов по сравнению с униграммами

In [ ]:
bigram_vectorized_x_train.shape

(62999, 461507)

"Признаков" объектов стало на порядок больше.

## Токенизация

Токенизировать - значит, поделить текст на части: слова, ключевые слова, фразы, символы и т.д., иными словами **токены**.

Самый наивный способ токенизировать текст - разделить с помощью функции `split()`. Но `split` упускает очень много всего, например, не отделяет пунктуацию от слов. Кроме этого, есть ещё много менее тривиальных проблем, поэтому лучше использовать готовые токенизаторы.

In [ ]:
import nltk # уже знакомая нам библиотека nltk
from nltk.tokenize import word_tokenize # готовый токенизатор библиотеки nltk

Чтобы использовать токенизатор ```word_tokenize```, нужно сначала скачать данные для nltk о пунктуации и стоп-словах.

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Применим токенизацию:

In [ ]:
sentence = 'Кто же победит на выборах в США: Трамп или Байден?'
word_tokenize(sentence)

['Кто',
 'же',
 'победит',
 'на',
 'выборах',
 'в',
 'США',
 ':',
 'Трамп',
 'или',
 'Байден',
 '?']

Сравните с использованием ```split()```:

In [ ]:
sentence.split()

['Кто',
 'же',
 'победит',
 'на',
 'выборах',
 'в',
 'США:',
 'Трамп',
 'или',
 'Байден?']

В nltk вообще есть довольно много токенизаторов:

In [ ]:
from nltk import tokenize
dir(tokenize)[:16]

['BlanklineTokenizer',
 'LineTokenizer',
 'MWETokenizer',
 'PunktSentenceTokenizer',
 'RegexpTokenizer',
 'ReppTokenizer',
 'SExprTokenizer',
 'SpaceTokenizer',
 'StanfordSegmenter',
 'TabTokenizer',
 'TextTilingTokenizer',
 'ToktokTokenizer',
 'TreebankWordTokenizer',
 'TweetTokenizer',
 'WhitespaceTokenizer',
 'WordPunctTokenizer']

Одни умеют выдавать индексы в строке для начала и конца каждого слова-токена:

In [ ]:
wh_tok = tokenize.WhitespaceTokenizer()
list(wh_tok.span_tokenize(sentence))

[(0, 3),
 (4, 6),
 (7, 14),
 (15, 17),
 (18, 25),
 (26, 27),
 (28, 32),
 (33, 38),
 (39, 42),
 (43, 50)]

Некторые токенизаторы ведут себя специфично:

In [ ]:
tokenize.TreebankWordTokenizer().tokenize("don't stop me")

['do', "n't", 'stop', 'me']

А некоторые -- вообще не для текста на естественном языке:

In [ ]:
tokenize.SExprTokenizer().tokenize("(a (b c)) d e (f)")

['(a (b c))', 'd', 'e', '(f)']

**Правильный токенизатор подбирается исходя из требований задачи!**

## Стоп-слова

**Стоп-слова** - это слова, которые часто встречаются практически в любом тексте и ничего интересного не говорят о конретном документе. Для модели это просто шум. А шум нужно убирать. По аналогичной причине убирают и пунктуацию.

In [ ]:
# импортируем стоп-слова из библиотеки nltk
from nltk.corpus import stopwords

# посмотрим на стоп-слова для русского языка
print(stopwords.words('russian'))

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

In [ ]:
noise = stopwords.words('russian')

Теперь нужно обучать нашу модель с учетом новых знаний про токенизацию и стоп-слова. 

Для этого мы можем собрать новый векторизатор, передав ему на вход:
* какие n-граммы нам нужны, параметр **ngram_range**;
* какой токенизатор мы используем, параметр **tokenizer**;
* какие у нас стоп-слова, параметр **stop_words**.

In [ ]:
# инициализируем умный векторайзер 
smart_vectorizer = CountVectorizer(ngram_range=(1, 1), stop_words=noise)

In [ ]:
# обучаем его и сразу применяем к x_train
smart_vectorized_x_train = smart_vectorizer.fit_transform(x_train)

In [ ]:
smart_vectorized_x_train.shape

(62999, 44348)

In [ ]:
list(smart_vectorizer.vocabulary_.items())[:10]

[('пришёл', 31452),
 ('мал', 17955),
 ('материал', 18213),
 ('ожидался', 23391),
 ('немного', 20921),
 ('прозрачный', 31910),
 ('товар', 39502),
 ('пришел', 31385),
 ('менее', 18398),
 ('недели', 20604)]

In [ ]:
# инициализируем и обучаем классификатор
clf = MultinomialNB()
clf.fit(smart_vectorized_x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
# применяем обученный векторайзер к тестовым данным
smart_vectorized_x_test = smart_vectorizer.transform(x_test)

In [ ]:
# получаем предсказания и выводим информацию о качестве
pred = clf.predict(smart_vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    neautral       0.58      0.65      0.61      9054
    negative       0.71      0.61      0.66      8966
    positive       0.82      0.84      0.83      8981

    accuracy                           0.70     27001
   macro avg       0.70      0.70      0.70     27001
weighted avg       0.70      0.70      0.70     27001



Получилось чуть хуже. 

Что ещё можно сделать?

## Лемматизация

**Лемматизация** – это сведение разных форм одного слова к начальной форме – **лемме**. Почему это хорошо?
* Во-первых, естественно рассматривать как отдельный признак каждое *слово*, а не каждую его отдельную форму.
* Во-вторых, некоторые стоп-слова стоят только в начальной форме, и без лематизации выкидываем мы только её.

Для русского есть хороший лемматизатор pymorphy. 

Стемминг (англ. stemming — находить происхождение) — это процесс нахождения основы слова для заданного исходного слова. Основа слова не обязательно совпадает с морфологическим корнем слова. 

### [Pymorphy](http://pymorphy2.readthedocs.io/en/latest/)
Это модуль на питоне, довольно быстрый и с кучей функций.

In [ ]:
# устанавливаем pymorphy2
!pip install pymorphy2

     |████████████████████████████████| 61kB 1.8MB/s 
     |████████████████████████████████| 8.2MB 4.9MB/s 


В pymorphy2 для морфологического анализа слов есть ```MorphAnalyzer()```:

In [ ]:
from pymorphy2 import MorphAnalyzer
pymorphy2_analyzer = MorphAnalyzer()

pymorphy2 работает с отдельными словами. Если дать ему на вход предложение - он его просто не лемматизирует, т.к. не понимает:

In [ ]:
sentence = 'Кто же победит на выборах в США: Трамп или Байден?'
sent = word_tokenize(sentence)

Лемматизируем слово "победит" из предложения ```sentence``` с помощью метода ```parse()```:

In [ ]:
ana = pymorphy2_analyzer.parse(sent[2])
ana

[Parse(word='победит', tag=OpencorporaTag('VERB,perf,tran sing,3per,futr,indc'), normal_form='победить', score=0.846153, methods_stack=((DictionaryAnalyzer(), 'победит', 2483, 9),)),
 Parse(word='победит', tag=OpencorporaTag('NOUN,inan,masc sing,nomn'), normal_form='победит', score=0.076923, methods_stack=((DictionaryAnalyzer(), 'победит', 34, 0),)),
 Parse(word='победит', tag=OpencorporaTag('NOUN,inan,masc sing,accs'), normal_form='победит', score=0.076923, methods_stack=((DictionaryAnalyzer(), 'победит', 34, 3),))]

Выведем его нормальную форму:

In [ ]:
ana[0].normal_form

'победить'

Нормализация предложения "вижу три села" может дать "видеть тереть сесть"

In [ ]:
sent2 = word_tokenize('вижу три села')

In [ ]:
ana2 = pymorphy2_analyzer.parse(sent2[2])
ana2[0].normal_form

'село'

## TF-IDF векторизация

`TfidfVectorizer` делает то же, что и `CountVectorizer`, но в качестве значений выдает **tf-idf** каждого слова.

Как считается tf-idf:

**TF (term frequency)** – относительная частотность слова в документе:
$$ TF(t,d) = \frac{n_{t}}{\sum_k n_{k}} $$

**IDF (inverse document frequency)** – обратная частота документов, в которых есть это слово:
$$ IDF(t, D) = \mbox{log} \frac{|D|}{|{d : t \in d}|} $$

Перемножаем их:
$$TFIDF(t, d, D) = TF(t,d) \times IDF(i, D)$$

Cмысл: если слово часто встречается в одном документе, но в целом по корпусу встречается в небольшом 
количестве документов, у него высокий TF-IDF.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

Действуем аналогично, как с ```CountVectorizer()```:

In [ ]:
# инициализируем векторизатор, в качестве переменных используем униграммы
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1))

In [ ]:
# обучаем его и сразу применяем к x_train
tfidf_vectorized_x_train = tfidf_vectorizer.fit_transform(x_train)

In [ ]:
# инициализируем и обучаем классификатор
clf = MultinomialNB()
clf.fit(tfidf_vectorized_x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
# применяем обученный векторизатор к тестовым данным
tfidf_vectorized_x_test = tfidf_vectorizer.transform(x_test)

# получаем предсказания и выводим информацию о качестве
pred = clf.predict(tfidf_vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    neautral       0.59      0.67      0.63      9054
    negative       0.72      0.63      0.67      8966
    positive       0.84      0.84      0.84      8981

    accuracy                           0.71     27001
   macro avg       0.72      0.71      0.71     27001
weighted avg       0.72      0.71      0.71     27001



Иногда пунктуация бывает и не шумом - главное отталкиваться от задачи. Что будет если вообще не убирать пунктуацию? На примере с твитами хорошо было бы видно, что пунктуация работает хорошо ))

In [ ]:
# инициализируем умный векторайзер stop-words НЕ ИСПОЛЬЗУЕМ!
alternative_tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1), 
                                               tokenizer=word_tokenize)

# обучаем его и сразу применяем к x_train
alternative_tfidf_vectorized_x_train = alternative_tfidf_vectorizer.fit_transform(x_train)

# инициализируем и обучаем классификатор
clf = MultinomialNB()
clf.fit(alternative_tfidf_vectorized_x_train, y_train)

# применяем обученный векторайзер к тестовым данным
alternative_tfidf_vectorized_x_test = alternative_tfidf_vectorizer.transform(x_test)

# получаем предсказания и выводим информацию о качестве
pred = clf.predict(alternative_tfidf_vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    neautral       0.59      0.67      0.63      9054
    negative       0.72      0.64      0.68      8966
    positive       0.85      0.84      0.84      8981

    accuracy                           0.71     27001
   macro avg       0.72      0.71      0.72     27001
weighted avg       0.72      0.71      0.72     27001



Посмотрим, как один из супер-значительных токенов справится с классификацией безо всякого машинного обучения:

In [ ]:
cool_token = 'плохо'
pred = ['positive' if cool_token in review else 'negative' for review in x_test]
print(classification_report(pred, y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

    neautral       0.00      0.00      0.00         0
    negative       0.94      0.33      0.49     25808
    positive       0.02      0.14      0.03      1193

    accuracy                           0.32     27001
   macro avg       0.32      0.16      0.17     27001
weighted avg       0.90      0.32      0.47     27001



## Символьные n-граммы

В некоторых задачах в качестве признаков могут быть использщованы, n-граммы символов. Для этого необходимо установить в ```CountVectorizer()``` параметр ```analyzer = 'char'```, то есть анализировать символы.

In [ ]:
# инициализируем векторайзер для символов
char_vectorizer = CountVectorizer(analyzer='char', ngram_range=(3, 6))

# обучаем его и сразу применяем к x_train
char_vectorized_x_train = char_vectorizer.fit_transform(x_train)

# инициализируем и обучаем классификатор
clf = MultinomialNB()
clf.fit(char_vectorized_x_train, y_train)

# применяем обученный векторайзер к тестовым данным
char_vectorized_x_test = char_vectorizer.transform(x_test)

# получаем предсказания и выводим информацию о качестве
pred = clf.predict(char_vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    neautral       0.59      0.71      0.64      9054
    negative       0.73      0.62      0.67      8966
    positive       0.87      0.83      0.85      8981

    accuracy                           0.72     27001
   macro avg       0.73      0.72      0.72     27001
weighted avg       0.73      0.72      0.72     27001



Cимвольные n-граммы используются, например, для задачи определения языка. Ещё одна замечательная особенность признаков-символов - для них не нужна токенизация и лемматизация, можно использовать такой подход для языков, у которых нет готовых анализаторов.

## Задание 5

Применим полученные выше навыки и решим задачу анализа тональности отзывов. (Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`, он есть в папке `Data`).

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Обязательные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)
6. построение модели
7. оценка качества модели

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать параметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор. 

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

from sklearn.metrics import * 
from sklearn.model_selection import train_test_split

from sklearn import preprocessing

In [ ]:
data = pd.read_csv('women-clothing-accessories.csv', sep='\t', usecols=[0, 1])

In [ ]:
data.sample(10, random_state=20)

,review,sentiment
13112,Ужасно пошито. Очень жаль.,negative
86903,"90/75/100 рост 169 заказала xl, свитер сидит...",positive
24651,"разачерована ,чуть чуть другое ожидала ,не пон...",negative
27032,доставка 3 с половиной месяца,negative
46252,"В целом товар соответствует описанию, но штаны...",neautral
22496,что за непонятная тряпка ? с ржавыми дырками. ...,negative
39199,На фото изображён другой пиджак! Сравните дета...,neautral
2721,На поясе вдоль резинки имеются дырки!!!,negative
78682,"Ожидала,что будет немного подлиньше,но итак вр...",positive
59396,"Ожидала качество лучше ,по факту так себе ,мат...",neautral


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data.review, data.sentiment, train_size = 0.7)

In [ ]:
import nltk # уже знакомая нам библиотека nltk
from nltk.tokenize import word_tokenize # готовый токенизатор библиотеки nltk
!pip install pymorphy2
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords

from nltk import ngrams

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer # модель "мешка слов"
from sklearn.naive_bayes import MultinomialNB # наивный байесовский классификатор

nltk.download('stopwords')
nltk.download('punkt')

     |████████████████████████████████| 61kB 4.6MB/s 
     |████████████████████████████████| 8.2MB 9.2MB/s 
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
pm = MorphAnalyzer()
stop = stopwords.words('russian')

In [ ]:
def myTokenizer(sentence):

  #tokenize
  words = word_tokenize(sentence)

  lemmatized_words = list()
  # remove stop words and lemmatize
  for word in words:
    if not word in stop:
      lemmatized_words.append(pm.parse(word)[0].normal_form)

  return lemmatized_words

def make_report_df(report, vectorizer_name='no_name'):
  comment_types = ['neautral', 'negative', 'positive']
  score_types = ['precision', 'recall', 'f1-score']

  result = {'accuracy': report['accuracy']}

  for score in score_types:
    avg = 0
    for comment in comment_types:
      avg = avg + report[comment][score]
    avg = avg / len(comment_types)
    result[score] = avg

  return pd.DataFrame(data=result, index=[vectorizer_name])

In [ ]:
# будем писать сюда дф с результатами по векторизаторам
global_report_df = pd.DataFrame()

In [ ]:
vectorizer = CountVectorizer(lowercase=True, tokenizer=myTokenizer, ngram_range=(1,3))

vectorized_x_train = vectorizer.fit_transform(x_train)

model = MultinomialNB()
model.fit(vectorized_x_train, y_train)

vectorized_x_test = vectorizer.transform(x_test)

In [ ]:
# получаем предсказания и пишем информацию о качестве
pred = model.predict(vectorized_x_test)
report_word_ngram = classification_report(y_test, pred, output_dict=True)

In [ ]:
# добавляем дф с результатами в общиий дф
global_report_df = global_report_df.append(make_report_df(report_word_ngram, "word ngram"))

In [ ]:
# выяснилось, что параметр max_df в целом не влияет на качество предсказаниия
# по сути он должен исключать специфические для набора стоп слова
# видимо таких тут крайне мало

# параметр min_df не дает никакого прироста, т.к 
# term document frequency колеблется в районе 0.0005

# а вот параметр max_features лучше снизить, чтобы не переобучить модель

tfidf_vectorizer = TfidfVectorizer(lowercase=True, tokenizer=myTokenizer, ngram_range=(1, 3), max_features=150_000)

tfidf_vectorized_x_train = tfidf_vectorizer.fit_transform(x_train)

clf = MultinomialNB()
clf.fit(tfidf_vectorized_x_train, y_train)

tfidf_vectorized_x_test = tfidf_vectorizer.transform(x_test)

In [ ]:
# получаем предсказания и пишем информацию о качестве
pred = clf.predict(tfidf_vectorized_x_test)
report_tfidf = classification_report(y_test, pred, output_dict=True)

In [ ]:
# добавляем дф с результатами в общиий дф
global_report_df = global_report_df.append(make_report_df(report_tfidf, "tfidf"))

In [ ]:
# инициализируем векторайзер
char_vectorizer = CountVectorizer(lowercase=True, analyzer='char', ngram_range=(3, 6))

# обучаем его и сразу применяем к x_train
char_vectorized_x_train = char_vectorizer.fit_transform(x_train)

# инициализируем и обучаем классификатор
clf = MultinomialNB()
clf.fit(char_vectorized_x_train, y_train)

# применяем обученный векторайзер к тестовым данным
char_vectorized_x_test = char_vectorizer.transform(x_test)

In [ ]:
# получаем предсказания и пишем информацию о качестве
pred = clf.predict(char_vectorized_x_test)
report_char_ngram = classification_report(y_test, pred, output_dict=True)

In [ ]:
# добавляем дф с результатами в общиий дф
global_report_df = global_report_df.append(make_report_df(report_char_ngram, "char ngram"))

In [ ]:
global_report_df

,accuracy,precision,recall,f1-score
word ngram,0.716862,0.715097,0.715896,0.715213
tfidf,0.718862,0.721037,0.717839,0.718470
char ngram,0.720047,0.730402,0.718960,0.721461


## Бонус 1. Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах. 

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [ ]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **?а** - ноль или один символ **а**
* **+а** - один или более символов **а**
* **\*а** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа

Пример:
Выражению \*a?b. соответствуют последовательности a, ab, abc, aa, aac НО НЕ abb!

Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**: 
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [ ]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc') 
print(result)

['abcd', 'abca']


Вопрос на внимательность: почему нет abcx?

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

### split
разделяет строку по заданному шаблону


In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie') 
print(result)

['itsy', ' bitsy', ' teenie', ' weenie']


можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2) 
print(result)

['itsy', ' bitsy', ' teenie, weenie']


**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc')
print (result)

bbcbbc


**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

## Бонус 2: Word2Vec

Векторные модели, которые мы рассматривали до этого (BOW, мешок слов; TF-IDF), условно называются *счётными*. Они основываются на том, что так или иначе "считают" слова и их соседей, и на основе этого строят вектора для слов. 


Другой класс моделей, который повсеместно распространён на сегодняшний день, называется *предсказательными* моделями. Идея этих моделей заключается в использовании нейросетевых архитектур, которые "предсказывают" (а не считают) соседей для каждого слова.

Одной из самых известных таких моделей является `word2vec`. Технология основана на нейронной сети, предсказывающей вероятность встретить слово в заданном контексте. Этот инструмент был разработан группой исследователей Google в 2013 году, руководителем проекта был Томаш Миколов (сейчас работает в Facebook). Вот две самые главные статьи:

* [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)
* [Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/abs/1310.4546)


Полученные таким образом вектора называются *распределенными представлениями слов*, вложениями или **эмбеддингами**.

### Как это обучается?
Мы задаём вектор для каждого слова с помощью матрицы $w$ и вектор контекста с помощью матрицы $W$. По сути, word2vec является обобщающим названием для двух архитектур Skip-Gram и Continuous Bag-Of-Words (CBOW).  

![](https://www.researchgate.net/profile/Daniel_Braun6/publication/326588219/figure/fig1/AS:652185784295425@1532504616288/Continuous-Bag-of-words-CBOW-CB-and-Skip-gram-SG-training-model-illustrations.png)

**CBOW** предсказывает текущее слово, исходя из окружающего его контекста. 

**Skip-gram**, наоборот, использует текущее слово, чтобы предугадывать окружающие его слова. 

### Как это работает?
Word2vec принимает большой текстовый корпус в качестве входных данных и сопоставляет каждому слову вектор, выдавая координаты слов на выходе. Сначала он создает словарь, «обучаясь» на входных текстовых данных, а затем вычисляет векторное представление слов. 

Векторное представление основывается на *контекстной близости*: слова, встречающиеся в тексте рядом с одинаковыми словами (а следовательно, согласно дистрибутивной гипотезе, имеющие схожий смысл), в векторном представлении будут иметь близкие координаты векторов-слов. Для вычисления близости слов используется косинусное расстояние между их векторами.


С помощью дистрибутивных векторных моделей можно строить семантические пропорции (они же аналогии: А относится к B так же, как C относится к D) и решать примеры:

* *король: мужчина = королева: женщина* 
 $\Rightarrow$ 
* *король - мужчина + женщина = королева*

![w2v](https://cdn-images-1.medium.com/max/2600/1*sXNXYfAqfLUeiDXPCo130w.png)

Для слов нет лейблов, но мы используем слова в контексте для сбора обучающей выборки.

## Skip gram

(Предсказание контекста по слову, один из основных параметров - windows_size)

![Замещающий текст](http://mccormickml.com/assets/word2vec/training_data.png)

1. Представляем корпус текста в формате One-hot encoding, подаем вектор на вход нейросети

2. В качестве активации последнего слоя используем softmax -> переходим в пространство вероятностей (как будто задача классификации с очень большим количеством классов)

3. Предсказываем слово контекста по максимальной вероятности

![Замещающий текст](https://miro.medium.com/max/875/0*FD_ZSVKFywSg-CJM.png)

Модель хорошо работает с небольшим количеством тренировочных данных



## CBOW

![](https://iksinc.files.wordpress.com/2015/04/screen-shot-2015-04-12-at-10-58-21-pm.png)

Тренируется быстрее, чем SkipGram, лучше точность на редких словах


### Проблемы
Невозможно установить тип семантических отношений между словами: синонимы, антонимы и т.д. будут одинаково близки, потому что обычно употребляются в схожих контекстах. Поэтому близкие в векторном пространстве слова называют *семантическими ассоциатами*. Это значит, что они семантически связаны, но как именно — непонятно.


## RusVectōrēs


На сайте [RusVectōrēs](https://rusvectores.org/ru/) собраны предобученные на различных данных модели для русского языка, а также можно поискать наиболее близкие слова к заданному, посчитать семантическую близость нескольких слов и порешать примеры с помощью «калькулятора семантической близости».


Для других языков также можно найти предобученные модели — например, модели [fastText](https://fasttext.cc/docs/en/english-vectors.html) и [GloVe](https://nlp.stanford.edu/projects/glove/).

## Gensim

Использовать предобученную модель эмбеддингов или обучить свою можно с помощью библиотеки `gensim`. Вот [ее документация](https://radimrehurek.com/gensim/models/word2vec.html).

### Как использовать готовую модель

Модели word2vec бывают разных форматов:

* .vec.gz — обычный файл (текстовый)
* .bin.gz — бинарный файл

Загружаются они с помощью одного и того же класса `KeyedVectors`, меняется только параметр `binary` у функции `load_word2vec_format`. 

Если же эмбеддинги обучены **не** с помощью word2vec, то для загрузки нужно использовать функцию `load`. Т.е. **для загрузки предобученных эмбеддингов *glove, fasttext, bpe* и любых других нужна именно она**.

Скачаем с RusVectōrēs модель для русского языка, обученную на НКРЯ образца 2015 г. 

In [ ]:
import urllib.request # библиотека для скачивания данных
import gensim # библиотека для загрузки и использвоания моделй w2v
from gensim.models import word2vec # непосредственно методы w2v

In [ ]:
# скачиваем модель ruscorpora_mystem_cbow_300 с сайта rusvectores
# 300 - размерность вектора embeddings для слов

urllib.request.urlretrieve("http://rusvectores.org/static/models/rusvectores2/ruscorpora_mystem_cbow_300_2_2015.bin.gz", "ruscorpora_mystem_cbow_300_2_2015.bin.gz")

('ruscorpora_mystem_cbow_300_2_2015.bin.gz',
 <http.client.HTTPMessage at 0x7fc2cfc21048>)

Загружаем скачанную модель. Обратите внимание, что мы скачали бинарный файл (.bin.gz), поэтому у функции load_word2vec_format() параметр binary=True

In [ ]:
model_path = 'ruscorpora_mystem_cbow_300_2_2015.bin.gz'

model_ru = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)

Посмотрим на ближайших соседей следующей группы слов:

In [ ]:
words = ['день_S', 'ночь_S', 'человек_S', 'семантика_S', 'биткоин_S']

Частеречные тэги (например, _S, тег части речи слова) нужны, поскольку это специфика скачанной модели - она была натренирована на словах, размеченных по частям речи (и лемматизированных). 

**NB!** В названиях моделей на `rusvectores` указано, какой тегсет (набор обозначений тегов) они используют (mystem, upos и т.д.)

Попросим у модели 10 ближайших соседей для каждого слова и косинусные близости для каждого:

In [ ]:
for word in words:
    # есть ли слово в модели? 
    if word in model_ru:
        print(word)
        # смотрим на вектор слова (его размерность 300, смотрим на первые 10 чисел)
        print(model_ru[word][:10])
        # выдаем 10 ближайших соседей слова:
        for word, sim in model_ru.most_similar(positive=[word], topn=10):
            # слово + коэффициент косинусной близости
            print(word, ': ', sim)
        print('\n')
    else:
        # Увы!
        print('Увы, слова "%s" нет в модели!' % word)

день_S
[-0.02580778  0.00970898  0.01941961 -0.02332282  0.02017624  0.07275085
 -0.01444375  0.03316632  0.01242602  0.02833412]
неделя_S :  0.7165195941925049
месяц_S :  0.631048858165741
вечер_S :  0.5828739404678345
утро_S :  0.5676207542419434
час_S :  0.5605547428131104
минута_S :  0.5297019481658936
гекатомбеон_S :  0.4897990822792053
денек_S :  0.48224714398384094
полчаса_S :  0.48217129707336426
ночь_S :  0.478074848651886


ночь_S
[-0.00688948  0.00408364  0.06975466 -0.00959525  0.0194835   0.04057068
 -0.00994112  0.06064967 -0.00522624  0.00520327]
вечер_S :  0.6946247816085815
утро_S :  0.57301926612854
ноченька_S :  0.5582467317581177
рассвет_S :  0.5553582906723022
ночка_S :  0.5351512432098389
полдень_S :  0.5334426164627075
полночь_S :  0.478694349527359
день_S :  0.4780748784542084
сумерки_S :  0.4390218257904053
фундерфун_S :  0.4340824782848358


человек_S
[ 0.02013756 -0.02670703 -0.02039861 -0.05477146  0.00086402 -0.01636335
  0.04240306 -0.00025525 -0.14045681 

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Найдем похожесть пары слов функцией ```similarity()``` (там используется косинусная мера схожести):

In [ ]:
print(model_ru.similarity('человек_S', 'обезьяна_S'))

0.23895611


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


У загруженной модели много различных функций. Например, можно решать задачи на семантическую близость.

Что получится, если вычесть из пиццы Италию и прибавить Сибирь?

Для решения примера в качестве параметров метода ```most_similar()``` необходимо передать:
* positive — вектора, которые мы складываем
* negative — вектора, которые вычитаем

*Замечание:* не забываем взять самый близкий элемент, для этого необходимо указать ```[0][0]```.

Что получится, если вычесть из пиццы Италию и прибавить Сибирь?

In [ ]:
print(model_ru.most_similar(negative=[ 'италия_S'], positive=['пицца_S','сибирь_S'])[0][0])

пельмень_S


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
print(model_ru.most_similar(positive=['футбол_S', 'хоккей_S'], negative=['россия_S'])[0][0])

волейбол_S


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


**Задание.** Придумайте и проверьте с помощью метода `most_similar` несколько аналогий

Метод ```doesnt_match()``` находит "лишнее слово" в группе слов:

In [ ]:
model_ru.doesnt_match('пицца_S пельмень_S хот-дог_S ананас_S'.split())

/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'ананас_S'

**Задание.** Придумайте и проверьте с помощью метода `doesnt_match` несколько последовательностей с лишними словами

### Как обучить свою модель

В качестве обучающих данных возьмем размеченные и неразмеченные отзывы о фильмах (датасет взят с Kaggle).

In [ ]:
# скачиваем датасет
! wget https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/train/unlabeledTrainData.tsv

--2020-11-07 07:18:46--  https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/train/unlabeledTrainData.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67281491 (64M) [text/plain]
Saving to: ‘unlabeledTrainData.tsv’

unlabeledTrainData. 100%[===================>]  64.16M  65.1MB/s    in 1.0s    

2020-11-07 07:18:47 (65.1 MB/s) - ‘unlabeledTrainData.tsv’ saved [67281491/67281491]



Загрузим датасет в датафрейм и посмотрим на него, делаем это с помощью уже привычной библиотеки **pandas**:

In [ ]:
# считываем данные в формате csv
data = pd.read_csv("unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

len(data)

50000

In [ ]:
data.head()

,id,review
0,"""9999_0""","""Watching Time Chasers, it obvious that it was..."
1,"""45057_0""","""I saw this film about 20 years ago and rememb..."
2,"""15561_0""","""Minor Spoilers<br /><br />In New York, Joan B..."
3,"""7161_0""","""I went to see this film with a great deal of ..."
4,"""43971_0""","""Yes, I agree with everyone on this site this ..."


In [ ]:
data.iloc[10]['review']

'"After reading the comments for this movie, I am not sure whether I should be angry, sad or sickened. Seeing comments typical of people who a)know absolutely nothing about the military or b)who base everything they think they know on movies like this or on CNN reports about Abu-Gharib makes me wonder about the state of intellectual stimulation in the world.<br /><br />At the time I type this the number of people in the US military: 1.4 million on Active Duty with another almost 900,000 in the Guard and Reserves for a total of roughly 2.3 million.<br /><br />The number of people indicted for abuses at at Abu-Gharib: Currently less than 20<br /><br />That makes the total of people indicted .00083% of the total military. Even if you indict every single military member that ever stepped in to Abu-Gharib, you would not come close to making that a whole number. <br /><br />The flaws in this movie would take YEARS to cover. I understand that it\'s supposed to be sarcastic, but in reality, th

Нам необходимо отчистить данные от лишнего: убрать ссылки, html-разметку и небуквенные символы. Затем нужно привести все к нижнему регистру и токенизировать. 

На выходе мы хотим получить массив из предложений, каждое из которых представляет собой массив слов.

Импортируем необходимые библиотеки и методы (некоторые уже были испортированы ранее, но для полноты картины оставим их):

In [ ]:
import nltk.data # библиотека Natural Language Toolkit
import re   # библиотека для регулярных выражений
from bs4 import BeautifulSoup # библиотека для парсинга xml
from nltk.corpus import stopwords # стоп-слова из NLTK
from nltk.tokenize import sent_tokenize, RegexpTokenizer  # токенизаторы из NLTK
nltk.download('punkt') # пунктуация для правильной работы токенизатора

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

Функции для очистки данных:


In [ ]:
def review_to_wordlist(review, remove_stopwords=False):
    # убираем ссылки вне тегов
    review = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", " ", review)
    # достаем сам текст
    review_text = BeautifulSoup(review, "lxml").get_text()
    # оставляем только буквенные символы
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    # приводим к нижнему регистру и разбиваем на слова по символу пробела
    words = review_text.lower().split()
    if remove_stopwords:
      # убираем стоп-слова
        stops = stopwords.words("english")
        words = [w for w in words if not w in stops]
    return(words)

def review_to_sentences(review, tokenizer, remove_stopwords=False):
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords))
    return sentences

Проходим по всему датасету и парсим написанной выше функцией  текст в списки слов, удаляя при этом лишнее:

In [ ]:
sentences = []  

print("Parsing sentences from training set...")
for review in data["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set...


/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:273: UserWarning: "b'..'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


Посмотрим, что получилось:

In [ ]:
print(len(sentences))
print(sentences[0])

528987
['watching', 'time', 'chasers', 'it', 'obvious', 'that', 'it', 'was', 'made', 'by', 'a', 'bunch', 'of', 'friends']


In [ ]:
# это понадобится нам позже для обучения другой модели эмбеддингов 

with open('clean_text.txt', 'w') as f:
    for s in sentences[:5000]:
        f.write(' '.join(s))
        f.write('\n')

Обучаем и сохраняем модель. 


Основные параметры:
* данные должны быть итерируемым объектом 
* size — размер вектора, 
* window — размер окна наблюдения,
* min_count — мин. частотность слова в корпусе,
* sg — используемый алгоритм обучения (0 — CBOW, 1 — Skip-gram),
* sample — порог для downsampling'a высокочастотных слов,
* workers — количество потоков,
* alpha — learning rate,
* iter — количество итераций,
* max_vocab_size — позволяет выставить ограничение по памяти при создании словаря (т.е. если ограничение превышается, то низкочастотные слова будут выбрасываться). Для сравнения: 10 млн слов = 1Гб RAM.

**NB!** Обратите внимание, что тренировка модели не включает препроцессинг! Это значит, что избавляться от пунктуации, приводить слова к нижнему регистру, лемматизировать их, проставлять частеречные теги придется до тренировки модели (если, конечно, это необходимо для вашей задачи). Т.е. в каком виде слова будут в исходном тексте, в таком они будут и в модели.

In [ ]:
print("Training model...")
# обучаем модель с векторами размерности 300, длиной окна 10
%time model_en = word2vec.Word2Vec(sentences, workers=4, size=300, min_count=10, window=10, sample=1e-3)

Training model...
CPU times: user 4min 24s, sys: 781 ms, total: 4min 25s
Wall time: 2min 20s


Смотрим, сколько в модели слов.

In [ ]:
print(len(model_en.wv.vocab))

28308


Попробуем оценить модель вручную, порешав примеры. Несколько дано ниже, попробуйте придумать свои.

In [ ]:
print(model_en.wv.most_similar(positive=["woman", "actor"], negative=["man"], topn=1))
print(model_en.wv.most_similar(positive=["dogs", "man"], negative=["dog"], topn=1))

print(model_en.wv.most_similar("usa", topn=3))

print(model_en.wv.doesnt_match("comedy thriller western novel".split()))

[('actress', 0.7793270945549011)]
[('men', 0.6503796577453613)]
[('europe', 0.7524359226226807), ('germany', 0.7389851808547974), ('north', 0.7247494459152222)]
novel


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


### Как дообучить существующую модель

При тренировке модели "с нуля" веса инициализируются случайно, однако, можно использовать для инициализации векторов веса из предобученной модели, таким образом как бы дообучая ее.

Сначала посмотрим близость какой-нибудь пары слов в имеющейся модели, чтобы потом сравнить результат с дообученной.

In [ ]:
model_en.wv.similarity('lion', 'rabbit')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.287468

В качестве дополнительных данных для обучения возьмем английский текст «Алисы в Зазеркалье».

In [ ]:
! wget https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/train/alice.txt

--2020-11-07 07:27:28--  https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/train/alice.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 167631 (164K) [text/plain]
Saving to: ‘alice.txt’

alice.txt           100%[===================>] 163.70K  --.-KB/s    in 0.04s   

2020-11-07 07:27:28 (3.80 MB/s) - ‘alice.txt’ saved [167631/167631]



In [ ]:
with open("alice.txt", 'r', encoding='utf-8') as f:
    text = f.read()

# убираем переносы строк, токенизируем текст

text = re.sub('\n', ' ', text)
sents = sent_tokenize(text)

punct = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~„“«»†*—/\-‘’'
clean_sents = []

# убираем всю пунктуацию и делим текст на слова по пробелу
for sent in sents:
    s = [w.lower().strip(punct) for w in sent.split()]
    clean_sents.append(s)
    
print(clean_sents[:2])

[['through', 'the', 'looking-glass', 'by', 'lewis', 'carroll', 'chapter', 'i', 'looking-glass', 'house', 'one', 'thing', 'was', 'certain', 'that', 'the', 'white', 'kitten', 'had', 'had', 'nothing', 'to', 'do', 'with', 'it', '', 'it', 'was', 'the', 'black', 'kitten’s', 'fault', 'entirely'], ['for', 'the', 'white', 'kitten', 'had', 'been', 'having', 'its', 'face', 'washed', 'by', 'the', 'old', 'cat', 'for', 'the', 'last', 'quarter', 'of', 'an', 'hour', 'and', 'bearing', 'it', 'pretty', 'well', 'considering', 'so', 'you', 'see', 'that', 'it', 'couldn’t', 'have', 'had', 'any', 'hand', 'in', 'the', 'mischief']]


Чтобы дообучить модель, надо сначала ее сохранить, а потом загрузить. Все параметры тренировки (размер вектора, мин. частота слова и т.п.) будут взяты из загруженной модели, т.е. задать их заново нельзя.

**NB!** Дообучить можно только полную модель (сохраненные при обучении веса и параметры модели, то есть обект самой модели), а `KeyedVectors` (просто пары "слово - вектор") — нельзя. Поэтому сохранять модель нужно в соотвествующем формате. Подробнее о разнице [вот тут](https://radimrehurek.com/gensim/models/keyedvectors.html).

In [ ]:
model_path = "movie_reviews.model"

# так можно сохранить модель для последующего дообучения
print("Saving model...")
model_en.save(model_path)

Saving model...


In [ ]:
# загружаем нашу обученную модель и дообучаем на текстах "Алисы"

model = word2vec.Word2Vec.load(model_path)

model.build_vocab(clean_sents, update=True)
model.train(clean_sents, total_examples=model.corpus_count, epochs=5)

(96730, 150225)

In [ ]:
model.wv.similarity('lion', 'rabbit')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.30270875

Лев и кролик стали ближе друг к другу!

Можно нормализовать вектора, тогда модель будет занимать меньше RAM. Однако после этого её нельзя дотренировывать. Здесь используется L2-нормализация: вектора нормализуются так, что если сложить квадраты всех элементов вектора, в сумме получится 1. 

Кроме того, сохраним не полные вектора, а `KeyedVectors`.

In [ ]:
model.init_sims(replace=True)
model_path = "movies_alice.bin"

print("Saving model...")
model_en.wv.save_word2vec_format(model_path, binary=True)

Saving model...


## Оценка

Задача обучения модели w2v - это usupervised задача (обучение без учителя), "правильных" ответов нет, поэтому нельзя вычислить некую метрику качества, чтобы сравнить две модели между собой или просто по значению одной метрики сказать, насколько хороша полученная модель. 

Тем не менее, существуют специальные выборки для оценки качества дистрибутивных моделей. Основных два: один измеряет точность решения задач на аналогии (пример про Россию и пельмени), а второй используется для оценки коэффициента семантической близости. 

### Аналогии

Другая популярная задача для "внутренней" оценки называется задачей поиска аналогий. Как мы уже разбирали выше, с помощью простых арифметических операций мы можем модифицировать значение слова. Если заранее собрать набор слов-модификаторов, а также слов, которые мы хотим получить в результаты модификации, то на основе подсчёта количества "попаданий" в желаемое слово мы можем оценить, насколько хорошо работает модель.

В качестве слов-модификаторов мы можем использовать семантические аналогии. Скажем, если у нас есть некоторое отношение "страна-столица", то для оценки модели мы можем использовать пары наподобие "Россия-Москва", "Норвегия-Осло", и т.д. Выборка будет выглядеть следующм образом:

| слово 1    | слово 2    | отношение     | 
|------------|------------|---------------|
| Россия     | Москва     | страна-столица|  
| Норвегия   | Осло       | страна-столица|

Рассматривая случайные две пары из этого набора, мы хотим, имея триплет (Россия, Москва, Норвегия), получить слово "Осло", т.е. найти такое слово, которое будет находиться в том же отношении со словом "Норвегия", как "Россия" находится с Москвой. 

Выборки для русского языка можно скачать на странице с моделями на RusVectores. Посчитаем качество нашей модели НКРЯ на выборке про аналогии:

In [ ]:
! wget https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/evaluation/ru_analogy_tagged.txt

In [ ]:
with open('ru_analogy_tagged.txt','r') as f:
  data = f.readlines()
  print (data[:10])

In [ ]:
res = model_ru.accuracy('ru_analogy_tagged.txt')

In [ ]:
print(res[4]['incorrect'][:10])

### Word Similarity

Этот метод заключается в том, чтобы оценить, насколько представления о семантической близости слов в модели соотносятся с "представлениями" людей.

| слово 1    | слово 2    | близость | 
|------------|------------|----------|
| кошка      | собака     | 0.7      |  
| чашка      | кружка     | 0.9      |       

Для каждой пары слов из заранее заданного датасета мы можем посчитать косинусное расстояние, и получить список таких значений близости. При этом у нас уже есть список значений близостей, сделанный людьми. Мы можем сравнить эти два списка и понять, насколько они похожи (например, посчитав корреляцию). Эта мера схожести должна говорить о том, насколько модель хорошо моделирует расстояния до слова.


## Бонус 3. FastText

FastText использует не только эмбеддинги слов, но и эмбеддинги n-грам. В корпусе каждое слово автоматически представляется в виде набора символьных n-грамм. 

Скажем, если мы установим n=3, то вектор для слова "where" будет представлен суммой векторов следующих триграм: "<wh", "whe", "her", "ere", "re>" (где "<" и ">" символы, обозначающие начало и конец слова). 

Благодаря этому мы можем также получать вектора для слов, отсутствуюших в словаре, а также эффективно работать с текстами, содержащими ошибки и опечатки.

* [Статья](https://aclweb.org/anthology/Q17-1010)
* [Сайт](https://fasttext.cc/)
* [Тьюториал](https://fasttext.cc/docs/en/support.html)
* [Вектора для 157 языков](https://fasttext.cc/docs/en/crawl-vectors.html)
* [Вектора, обученные на википедии](https://fasttext.cc/docs/en/pretrained-vectors.html) (отдельно для 294 разных языков)
* [Репозиторий](https://github.com/facebookresearch/fasttext)

Есть библиотека `fasttext` для питона (с готовыми моделями можно работать и через `gensim`).

In [ ]:
# чтобы установить fasstext, можно склонировать его с репозитория 
! git clone https://github.com/facebookresearch/fastText.git
! pip3 install fastText/.

Cloning into 'fastText'...
remote: Enumerating objects: 3854, done.
remote: Total 3854 (delta 0), reused 0 (delta 0), pack-reused 3854
Receiving objects: 100% (3854/3854), 8.22 MiB | 33.16 MiB/s, done.
Resolving deltas: 100% (2417/2417), done.
Processing ./fastText
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3015942 sha256=c15164321c84a239ec4e0c4964b1f90d43fc819f037f9ef2bf4fe350286b2c54
  Stored in directory: /tmp/pip-ephem-wheel-cache-755y22wk/wheels/a1/9f/52/696ce6c5c46325e840c76614ee5051458c0df10306987e7443
Successfully built fasttext


In [ ]:
import fasttext

ft_model = fasttext.train_unsupervised('clean_text.txt', minn=3, maxn=4, dim=300)

In [ ]:
ft_model.get_nearest_neighbors('actor')

[(0.9999606013298035, 'actors'),
 (0.9999364018440247, 'attractive'),
 (0.9999338984489441, 'fact'),
 (0.9999316334724426, 'actual'),
 (0.9999226331710815, 'display'),
 (0.9999191761016846, 'terrific'),
 (0.9999188780784607, 'battle'),
 (0.9999170899391174, 'israel'),
 (0.9999163746833801, 'british'),
 (0.9999160170555115, 'predator')]

In [ ]:
ft_model.get_analogies("woman", "man", "actor")

[(0.999938428401947, 'act'),
 (0.9998956918716431, 'exactly'),
 (0.9998955726623535, 'actress'),
 (0.999885082244873, 'seemingly'),
 (0.9998830556869507, 'terrible'),
 (0.9998824596405029, 'surprisingly'),
 (0.9998821020126343, 'believable'),
 (0.9998811483383179, 'double'),
 (0.9998807907104492, 'written'),
 (0.9998795986175537, 'cable')]

In [ ]:
ft_model.get_nearest_neighbors('actr')

[(0.9999391436576843, 'act'),
 (0.9998903274536133, 'actors'),
 (0.9998863339424133, 'actor'),
 (0.9998792409896851, 'actress'),
 (0.9998623728752136, 'single'),
 (0.9998517632484436, 'actual'),
 (0.9998226761817932, 'terrible'),
 (0.9998196363449097, 'exact'),
 (0.9998190402984619, 'plot'),
 (0.9998172521591187, 'wrong')]

In [ ]:
ft_model.get_nearest_neighbors('moviegeek')

[(0.9999324679374695, 'reviews'),
 (0.9999246597290039, 'review'),
 (0.9999151825904846, 'recommended'),
 (0.9999132752418518, 'rented'),
 (0.9998916387557983, 'waste'),
 (0.999889075756073, 'movie'),
 (0.9998835921287537, 'thank'),
 (0.9998812079429626, 'not'),
 (0.9998751878738403, 'watchable'),
 (0.9998645782470703, 'only')]

Дополнение: https://github.com/dipanjanS/text-analytics-with-python/tree/master/New-Second-Edition